In [2]:
%run calibration_Cube_v4.py
%matplotlib

Using matplotlib backend: Qt5Agg


### load data and define parameter

In [8]:
# initial parameter required
# pixel for SZ 10-12
name_RoI = ['Dual+MY']
name_dyes = ['Dual+MY']
name_single = ['Pt-TPTBP+MY', 'Pd-TPTBP+MY']
unit = 'hPa'
analyte = 'O2'

# pixel - selection from image cube shape: px(y/x) only dualindicator (name_dyes)    
# TFPP
# dark adaption
# pixel_rot = [[(200, 1597), (630, 1597), (630, 2047), (200, 2047)]]
# light adaption
# pixel_rot = [[(200, 1537), (630, 1537), (630, 2047), (200, 2047)]]

# TPTBP
# dark adaption
# pixel_rot = [[(200, 150), (630, 150), (630, 600), (200, 600)]]
# light adaption
pixel_rot = [[(200, 100), (630, 100), (630, 550), (200, 550)]]

print('Pixel size analyzed:')
print('\t', name_dyes[0], (pixel_rot[0][1][0] - pixel_rot[0][0][0]) * (pixel_rot[0][2][1] - pixel_rot[0][0][1]), 'px')

# ----------------------------------
arg = {'px color': '#ffa34d', 'color Pt': '#ff8c00', 'color meas1': '#9e7a4d',
       'color meas2': '#05f0f0', 'figure size meas': (5,3), 'cmap': 'inferno',
       'fontsize': 13.,  # general information for plot
       'rotation': 180., # load testcube -> rotate the cube for better visualization
       'show bands': 120} # which band shall be plotted in testcube? 
                                # either tuple (for range) or float (for single band) or
                                # None for all

# information for fitting and linear unmixing - same as for calibration
arg_fit = {'fit range sensor': (660, 900), 'fit range ref': (490, 580),
           'fit concentration': (0, 1000), # which concentration points shall be analyzed 
           'range lc': (660, 900)} # crop the region for linear unmixing
bnds = ((0, None), (0, None))

# Saving option. Note that the saving of svg-Files requires a rather long time.
save_op = {'type': ['png', 'tiff'], 'dpi': 300.}

# directory to measurement / calibration / generic function / reference files
file_hdr = '20200622_rhizosphere/light_adaption/20200622_optodes_100ms_run2_cube_radiometric.hdr'
path_calibrationFit = '20200701_ID-SZ10-16_radiometric/TPTBP/output/multiIndicator/calibration/' 
file_generic = '20200701_ID-SZ10-16_radiometric/TPTBP/output/Generic_function/202086_Generic_function.txt'
path_res = '20200622_rhizosphere/output_fullcalibration20200806/TPTBP/'

Pixel size analyzed:
	 Dual+MY 193500 px


#### pre-test for pixel and individual file 

In [9]:
# test plot cube -> check whether selected pixel/area is within cube shape
ls_px = []
for p in pixel_rot:
    for pp in p:
        ls_px.append(pp)
corr.load_testcube(file_hdr=file_hdr, pixel_rot=ls_px, span_area=True, arg=arg) 

### Measurement evaluation

In [10]:
# measurement_evaluation
# %run calibration_Cube_v4.py
# ==================================================================
start_time = time.time()
[cube, prefactors, dic_iratio, dic_calib, dic_calc, dfig_im, dfigures, 
 dic_iratio_comb] = dualSensor_evaluation(file_meas=file_hdr, path_calib=path_calibrationFit, arg=arg,
                                          arg_fit=arg_fit, name_dyes=name_RoI, pixel_rot=pixel_rot,
                                          file_generic=file_generic, name_single=name_single, 
                                          bnds=bnds, unit=unit, save_op=save_op, path_res=path_res,
                                          analyte=analyte, ref_equal=True, cutoff=5., max_calib=False,
                                          ratiometric=True, plotting=True, saving=True, 
                                          save_RoI=False, value_check=False)
print('=================================')
print("Time of execution --- %s seconds " % (time.time() - start_time))

Time of execution --- 1350.3807199001312 seconds 


### open measurement evaluation

In [ ]:
%run calibration_Cube_v4.py
path_ = '20200622_rhizosphere/output_fullcalibration/multiIndicator/measurement/'
for i in glob(path_ + '/*.hdf5'):
    if 'run2' in i:
        [dict_header, singleID, dic_raw, dict_calib, iratio1, iratio2, analyte1, analyte2, 
         dic_figures] = load_evaluation(file=i)